In [19]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [20]:
path = './'
num_model = 10

In [21]:
def process_result(dir, num_epochs=150):
    test_res_path = []
    for exp_idx in range(num_model):
        for file in glob.glob(dir+'/test/{}.txt'.format(str(exp_idx))):
            test_res_path.append(file)

    all_test_res = {}

    for exp_idx in range(num_model):
        f = open(test_res_path[exp_idx], 'r')
        data = f.read()
        all_test_res[exp_idx] = data
        f.close()

    train_loss_bce = {}
    train_loss_rest = {}
    test_acc = {}

    for exp_idx in range(num_model):
        train_loss_bce[exp_idx] = []
        train_loss_rest[exp_idx] = []
        test_acc[exp_idx] = []

        res = all_test_res[exp_idx].split('\n')

        for line_idx in range(len(res)-1):
            train = res[line_idx].split("|")[0]
            test = res[line_idx].split("|")[1]
            train_loss_bce[exp_idx].append(float(train.split(':')[1].split("+")[0]))
            train_loss_rest[exp_idx].append(float(train.split(':')[1].split("+")[1]))
            test_acc[exp_idx].append(float(test.split(':')[0]))

    train_loss_bce_mean = []
    train_loss_rest_mean = []
    test_acc_mean = []
    test_acc_best = []

    for epoch in range(num_epochs):
        train_loss_bce_temp_sum = 0
        train_loss_rest_temp_sum = 0
        test_acc_temp_sum = 0

        for exp_idx in range(num_model):
            test_acc_best.append(np.max(test_acc[exp_idx]))
            train_loss_bce_temp_sum += train_loss_bce[exp_idx][epoch]
            train_loss_rest_temp_sum += train_loss_rest[exp_idx][epoch]
            test_acc_temp_sum += test_acc[exp_idx][epoch]
        
        train_loss_bce_mean.append(train_loss_bce_temp_sum/num_model)
        train_loss_rest_mean.append(train_loss_rest_temp_sum/num_model)
        test_acc_mean.append(test_acc_temp_sum/num_model)

    plt.figure(figsize=(20,4))
    plt.subplot(1,3,1)
    plt.title('train bce loss')
    plt.plot(train_loss_bce_mean)
    plt.subplot(1,3,2)
    plt.title('train regularization loss')
    plt.plot(train_loss_rest_mean)
    plt.subplot(1,3,3)
    plt.title('mean test accuracy')
    plt.ylim(0,1)
    plt.plot(test_acc_mean)
    plt.savefig(dir + '/result.png')
    plt.close()

    with open(dir+'/test/performance.txt', "a") as file:
        file.write("---"*35)
        file.write('\n')
        file.write('Initial Mean Train Loss : ')
        file.write(str(train_loss_bce_mean[0]))
        file.write('+')
        file.write(str(train_loss_rest_mean[0]))
        file.write('\n')
        file.write('Final Mean Train Loss : ')
        file.write(str(train_loss_bce_mean[num_epochs-1]))
        file.write('+')
        file.write(str(train_loss_rest_mean[num_epochs-2]))
        file.write('\n')
        file.write('Best of Mean Test Acc : ')
        file.write(str(np.max(test_acc_mean)))
        file.write('\n')
        file.write('Best of Best Test Acc : ')
        file.write(str(np.max(test_acc_best)))
        file.write('\n')
        file.write('Mean of Best Test Acc : ')
        file.write(str(np.mean(test_acc_best)))
        file.write('\n')
        file.write('Std of Best Test Acc : ')
        file.write(str(np.std(test_acc_best)))
        file.write('\n')
    file.close()

    f = open(dir+'/mean_train_bce_loss.pkl', 'wb')
    pickle.dump(train_loss_bce_mean, f)
    f.close()
    f = open(dir+'/mean_train_rest_loss.pkl', 'wb')
    pickle.dump(train_loss_rest_mean, f)
    f.close()
    f = open(dir+'/mean_test_acc.pkl', 'wb')
    pickle.dump(test_acc_mean, f)
    f = open(dir+'/best_test_accs.pkl', 'wb')
    pickle.dump(test_acc_best, f)
    f.close()

In [22]:
ZBTB7A_opt = []
ZBTB7A_opt_200_epoch = []

for file in glob.glob(path+'ZBTB7A-multiple_exp_adam*') + glob.glob(path+'ZBTB7A-multiple_exp_sgd*'):
    if '200' in file:
        ZBTB7A_opt_200_epoch.append(file)
    else:
        ZBTB7A_opt.append(file)

In [7]:
ARID3A_opt = []
ARID3A_opt_200_epoch = []

for file in glob.glob(path+'ARID3A-multiple_exp_adam*') + glob.glob(path+'ARID3A-multiple_exp_sgd*'):
    if '200' in file:
        ARID3A_opt_200_epoch.append(file)
    else:
        ARID3A_opt.append(file)

In [8]:
ARID3A_opt_200_epoch

['./ARID3A-multiple_exp_adam_010_200_epoch',
 './ARID3A-multiple_exp_adam_001_200_epoch',
 './ARID3A-multiple_exp_adam_005_200_epoch',
 './ARID3A-multiple_exp_sgd_005_200_epoch',
 './ARID3A-multiple_exp_sgd_0005_200_epoch',
 './ARID3A-multiple_exp_sgd_001_200_epoch']

In [9]:
for file in ARID3A_opt_200_epoch:
    process_result(file)

In [16]:
ZBTB7A_opt_200_epoch = ZBTB7A_opt_200_epoch[2:]

In [18]:
for file in ZBTB7A_opt_200_epoch:
    process_result(file)